In [1]:
#!/usr/bin/env python3
"""
Read SO-CI output from Molpro.
Compare my eigenvalues/vectors with those printed by Molpro
KKI 5/25/2023
"""
import re, sys, copy, glob, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sys.path.insert(0, '../atomic_SOC')
import molpro_subs as mpr
import chem_subs as chem

pd.set_option('display.width', 1000)

In [2]:
fsoci = 'ac5z_hybB_r2p2444_lz.pro'
#soci = '../UMemphis/mgcl-equil-karl-block.pro'
fsoci = '../UMemphis/no.pro'
fsoci = '../UMemphis/mgcl_small4.pro'
print('SO-CI file: {:s}'.format(fsoci))

SO-CI file: ../UMemphis/mgcl_small4.pro


In [3]:
# read SO-CI
SOCI = mpr.fullmatSOCI(fsoci, hybrid=True)

Computational group = C2v
CASSCF states:
     3 Doublet
Replacing MRCI+Q energies by HLSDIAG values


In [4]:
# Compare eigenvalues
np.set_printoptions(suppress=True)
dif = SOCI.vals - SOCI.SOe.energies.Eshift.values
for irow, row in SOCI.SOe.energies.iterrows():
    print('{:3.0f}  {:9.2f}  {:9.2f}  {:9.2f}'.format(row.Nr, SOCI.vals[irow], row.Eshift, dif[irow]))

  1      -0.06      -0.06      -0.00
  2      -0.06      -0.06      -0.00
  3   26379.83   26379.83       0.00
  4   26379.83   26379.83       0.00
  5   26424.03   26424.03      -0.00
  6   26424.03   26424.03       0.00


In [5]:
# Compare weights
SOcompos = mpr.readSOcompos(fsoci)[0][-1]
diff = SOCI.vecsq*100 - SOcompos.pct
# columns of SOcompos.pct are SO states (rows are basis states)
def diff_ok(d, tol=0.02):
    s = np.abs(d).sum()
    if s < tol:
        return 'OK'
    else:
        return '*BAD*'
for ivec in range(SOCI.dimen):
    print(ivec)
    print('Mpro\t', np.round(SOcompos.pct[:,ivec], 2))
    print('mine\t', np.round(SOCI.vecsq[:,ivec]*100, 2))
    d = diff[:,ivec]
    print('diff\t', diff_ok(d), np.round(d, 2))


0
Mpro	 [  0. 100.   0.   0.   0.   0.]
mine	 [99.94  0.06  0.    0.    0.    0.  ]
diff	 *BAD* [ 99.94 -99.94   0.     0.     0.     0.  ]
1
Mpro	 [100.   0.   0.   0.   0.   0.]
mine	 [ 0.06 99.94  0.    0.    0.    0.  ]
diff	 *BAD* [-99.94  99.94   0.     0.     0.     0.  ]
2
Mpro	 [ 0.  0. 50.  0. 50.  0.]
mine	 [ 0.  0.  0. 50.  0. 50.]
diff	 *BAD* [  0.   0. -50.  50. -50.  50.]
3
Mpro	 [ 0.  0.  0. 50.  0. 50.]
mine	 [ 0.  0. 50.  0. 50.  0.]
diff	 *BAD* [  0.   0.  50. -50.  50. -50.]
4
Mpro	 [ 0.  0. 50.  0. 50.  0.]
mine	 [ 0.    0.   47.36  2.64 47.36  2.64]
diff	 *BAD* [ 0.    0.   -2.64  2.64 -2.64  2.64]
5
Mpro	 [ 0.  0.  0. 50.  0. 50.]
mine	 [ 0.    0.    2.64 47.36  2.64 47.36]
diff	 *BAD* [ 0.    0.    2.64 -2.64  2.64 -2.64]


In [6]:
# Compare eigenvectors
diff = SOCI.vec - SOCI.SOvec
def dot_ok(p, tol=0.001):
    # given a dot product from numpy.vdot(), return 
    #   'OK' if result is close to +/-1, else '*BAD*'
    r = np.absolute(p)
    re = np.real(p)
    x = abs(1 - abs(re))
    if x < tol:
        return 'OK'
    else:
        return '*BAD*'
for ivec in range(SOCI.dimen):
    print(ivec)
    provec = SOCI.SOvec[:,ivec]
    print('Mpro\t', np.round(provec, 3))
    myvec = SOCI.vec[:,ivec]
    print('mine\t', np.round(myvec, 3))
    p = np.vdot(provec, myvec)
    print('dot\t', dot_ok(p), np.round(p, 3))

0
Mpro	 [ 0.001+0.j     1.   +0.j     0.001+0.j    -0.   +0.j    -0.   -0.001j
 -0.   -0.j   ]
mine	 [ 1.   +0.j    -0.025+0.j    -0.   -0.j    -0.001+0.j     0.   +0.j
  0.   -0.001j]
dot	 *BAD* (-0.024+0j)
1
Mpro	 [ 1.   +0.j    -0.001+0.j    -0.   +0.j    -0.001+0.j     0.   +0.j
  0.   -0.001j]
mine	 [ 0.025+0.j     1.   +0.j     0.001+0.j    -0.   -0.j    -0.   -0.001j
 -0.   -0.j   ]
dot	 *BAD* (0.024+0j)
2
Mpro	 [-0.   +0.j     0.002+0.j    -0.707+0.j    -0.   +0.j     0.   +0.707j
  0.   -0.j   ]
mine	 [-0.002+0.j     0.   -0.j     0.   +0.j    -0.707-0.j    -0.   -0.j
 -0.   -0.707j]
dot	 *BAD* 0j
3
Mpro	 [ 0.002+0.j     0.   +0.j    -0.   +0.j     0.707+0.j     0.   +0.j
  0.   +0.707j]
mine	 [ 0.   +0.j     0.002+0.j    -0.707+0.j     0.   +0.j    -0.   +0.707j
  0.   -0.j   ]
dot	 *BAD* -0j
4
Mpro	 [-0.   +0.j    -0.   +0.j     0.707+0.j     0.   +0.j     0.   +0.707j
  0.   +0.j   ]
mine	 [ 0.   +0.j    -0.   +0.j     0.688-0.019j  0.102-0.126j  0.019+0.688j
 -0.126-0.102j

In [21]:
SOCI.vec

array([[ 0.99968855+0.j        ,  0.02490942+0.j        ,
        -0.00152412+0.j        ,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [-0.02490942+0.j        ,  0.99968855+0.j        ,
         0.        -0.j        ,  0.00152412+0.j        ,
        -0.        +0.j        ,  0.        +0.j        ],
       [-0.00002685-0.j        ,  0.00107738+0.j        ,
         0.        +0.j        , -0.70710596+0.j        ,
         0.68793239-0.0189363j , -0.15371816-0.05254686j],
       [-0.00107738+0.j        , -0.00002685-0.j        ,
        -0.70710596-0.j        ,  0.        +0.j        ,
         0.10202135-0.12642029j,  0.59206752-0.35080708j],
       [ 0.        +0.00002685j, -0.        -0.00107738j,
        -0.        -0.j        , -0.        +0.70710596j,
         0.0189363 +0.68793239j,  0.05254686-0.15371816j],
       [ 0.        -0.00107738j, -0.        -0.00002685j,
        -0.        -0.70710596j,  0.        -0.j        ,
        -

In [29]:
matso = SOCI.matrix
mdif = matso - np.conj(matso).T
for x in mdif.flatten():
    a = np.absolute(x)
    if a > 0:
        print(a)

In [30]:
fsoci

'../UMemphis/mgcl_small4.pro'